In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r /content/Prompt-Optimization-in-Hallucination-Detection
!git clone https://github.com/dunp7/Prompt-Optimization-in-Hallucination-Detection.git

Cloning into 'Math_solving_using_LLM'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 306 (delta 55), reused 57 (delta 24), pack-reused 216 (from 1)
Receiving objects: 100% (306/306), 979.18 KiB | 27.20 MiB/s, done.
Resolving deltas: 100% (208/208), done.


In [ ]:

!pip install huggingface_hub
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install -r /content/Prompt-Optimization-in-Hallucination-Detection/requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu124


# Libraries

In [ ]:
import random
import torch
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from huggingface_hub import login
import sys
import shutil
import os
sys.path.append("/content/Prompt-Optimization-in-Hallucination-Detection")
sys.path.append("/content/Prompt-Optimization-in-Hallucination-Detection/my_utils")


from generate_results import generate_answers, generate_labels, gen_responses_probs
from my_utils.data import load_ds, sample_ds, load_results




from huggingface_hub import login

token_comp = "your_hugging_face_token"
login(token= token_comp)


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
seed = 42
random.seed(seed)

cuda


# Load Datasets

In [ ]:
n_samples = 50


_, multiarith_val = load_ds("multiarith", seed)
_, gsm8k_val = load_ds("gsm8k", seed)
_, svamp_val = load_ds("svamp", seed)



multiarith_sample = sample_ds(multiarith_val, n_samples, seed, "multi-arith")
gsm8k_sample = sample_ds(gsm8k_val, n_samples, seed, "gsm8k")
svamp_sample = sample_ds(svamp_val, n_samples, seed, "svamp")

del multiarith_val, gsm8k_val, svamp_val

# datasets = [multiarith_sample, gsm8k_sample, svamp_sample]
datasets = [gsm8k_sample]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset: multi-arith
Dataset({
    features: ['question', 'final_ans', 'answers'],
    num_rows: 50
}) 

Dataset: gsm8k
Dataset({
    features: ['question', 'answer', 'answers'],
    num_rows: 50
}) 

Dataset: svamp
Dataset({
    features: ['ID', 'Body', 'Question', 'Equation', 'Answer', 'Type', 'question_concat', 'question', 'type', 'equation', 'id', 'answers'],
    num_rows: 50
}) 



In [ ]:
datasets[0]['question'][0]

'Five food companies sponsored a local food bank. Foster Farms donated 45 dressed chickens; American Summits donated twice the number of bottled water than the number of dressed chicken donated by Foster Farms; Hormel donated three times the number of dressed chickens that Foster Farms donated; Boudin Butchers donated one-third of the number of dressed chickens that Hormel donated; Del Monte Foods donated 30 fewer bottles of water than American Summits. How many food items did the companies donate in total?'

In [ ]:
### Generate answers LLM
models = {
    "falcon7b_instruct": "tiiuae/falcon-7b-instruct",
    "opt6_7b": "facebook/opt-6.7b",
    "opt_350m": "facebook/opt-350m",
    "opt1_3b": "facebook/opt-1.3b",
    "opt2_7b": "facebook/opt-2.7b",
    "gemma2_9b": "google/gemma-2-9b-it",
    "gemma2_2b": "google/gemma-2-2b-it",
    "llama2_7b": "meta-llama/Llama-2-7b-chat-hf",
    "llama2_13b": "meta-llama/Llama-2-13b-chat-hf",
    "mistral7b_instruct": "mistralai/Mistral-7B-Instruct-v0.2",
}


In [ ]:
# Prompts
prompts = {
    "default": None,
    "zero_shot_CoT": "Let’s think step by step.",
    "ite_ape": "We’ll approach this methodically, proceeding one step at a time.",
    "apo": "Carefully analyze the details and perform precise arithmetic operations step by step, ensuring to apply the correct mathematical principles for accurate calculations.",
    "pe2": "Carefully calculate step by step, considering all details, including unique counts and overlaps. Accurately apply arithmetic to find the numerical answer, ensuring logical operations are correctly followed."
}


In [ ]:
from copy import deepcopy
from tqdm import tqdm
# Function to construct a prompt
def take_prompt(initial_prompt, question):
    prompt_end = (
        "Therefore, the answer (arabic numerals) is "
    )

    return f"{initial_prompt}\n{question}. {prompt_end}"
def gen_answers(datasets, gen_model, gen_tokenizer, instruct_prompt, save_path):
  for dataset in datasets:
    dataset_copy = deepcopy(dataset)
    all_responses = []
    for i in tqdm((range(len(dataset))), desc= "Gen Anwers"):
      prompt = take_prompt(instruct_prompt, dataset_copy[i]['question'])
      response = gen_responses_probs(gen_model, gen_tokenizer, prompt, number_responses=1, temperature=0.1)['sequences'][0]
      response_text = gen_tokenizer.decode(response, skip_special_tokens=True)
      all_responses.append(response_text)

    dataset = dataset.add_column("generated_answers", all_responses)
    dataset.save_to_disk(save_path + dataset.info.description)

def save_to_google_drive(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    try:
        dest_data_path = os.path.join(destination_folder, 'data')
        if os.path.exists(dest_data_path):
            shutil.rmtree(dest_data_path)  # Clear existing folder
        shutil.copytree(source_folder, dest_data_path)
        print(f"Folder copied to {destination_folder}")
    except Exception as e:
        print(f"Error copying folder to Google Drive: {e}")

In [ ]:
import shutil
import os

# Generate answers
for model_name, model_path in models.items():
    print(f"Processing model: {model_name}...")

    # Load model and tokenizer
    gen_model = AutoModelForCausalLM.from_pretrained(
        model_path, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    gen_tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Iterate over prompts
    for prompt_name, chosen_prompt in prompts.items():
        print(f"  Using prompt: {prompt_name}")
        print(datasets[0].info.description)
        # Create output directory
        data_answer_path = f"data/{datasets[0].info.description}/{model_name}/{prompt_name}/"
        os.makedirs(data_answer_path, exist_ok=True)

        # Generate answers (ensure this function is defined)
        gen_answers(datasets, gen_model, gen_tokenizer, chosen_prompt, data_answer_path)

        # Save to GG Drive
        save_to_google_drive('/content/data', '/content/drive/My Drive/Prompt_Improvement')


    # Clean up memory
    del gen_model, gen_tokenizer
    torch.cuda.empty_cache()



Processing model: mistral7b_instruct...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  Using prompt: default
gsm8k


Gen Anwers: 100%|██████████| 50/50 [03:35<00:00,  4.32s/it]


Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

Folder copied to /content/drive/My Drive/Prompt_Improvement
  Using prompt: pe2
gsm8k


Gen Anwers: 100%|██████████| 50/50 [03:32<00:00,  4.26s/it]


Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

Folder copied to /content/drive/My Drive/Prompt_Improvement
